In [1]:
import sys, os
if '../' not in sys.path:
    sys.path.insert(0, '../')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr

from modules.licensing import LicensingData, License
from modules.realestate import RealEstateData
from modules.dynamics import *
from modules.io import read_json, write_json
from modules.locations import Location

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Read raw business license data

In [ ]:
# read and preprocess licensing data, then save to hdf
# licensing = LicensingData('../data/raw/')
# licensing.save()

# Preprocess business license data

In [2]:
# load property values
property_values_path  = '../data/processed/realestate.hdf'
mvalfah = pd.concat(RealEstateData(property_values_path), axis=1)

In [3]:
# load licenses and temporarily exclude those outsize zipcodes for which real estate values are unavailable
licensing_data_path = '../data/processed/licenses.hdf'
licenses = LicensingData.load(licensing_data_path)
licenses = licenses[licenses['ZIP CODE'].isin(mvalfah.columns)]

In [4]:
naming = {
    'ID': 'RECORD_ID',
    'LICENSE ID': 'LICENSE_ID',
    'ACCOUNT NUMBER': 'ACCOUNT_ID',
    'SITE NUMBER': 'SITE_ID',
    'LEGAL NAME': 'NAME',
    'DOING BUSINESS AS NAME': 'DBA',
    'ZIP CODE': 'ZIP',
    'LICENSE CODE': 'CODE',
    'LICENSE DESCRIPTION': 'CODE_DESCRIPTION',
    'BUSINESS ACTIVITY ID': 'ACTIVITY_ID',
    'BUSINESS ACTIVITY': 'ACTIVITY',
    'LICENSE NUMBER': 'LICENSE_NUMBER',
    'APPLICATION TYPE': 'APPLICATION_TYPE',
    'LICENSE STATUS': 'STATUS',
    'DATE ISSUED': 'DATE_ISSUED',
    'LICENSE STATUS CHANGE DATE': 'DATE_CHANGED',    
    'LICENSE TERM START DATE': 'DATE_START', 
    'LICENSE TERM EXPIRATION DATE': 'DATE_END',
    'APPLICATION REQUIREMENTS COMPLETE': 'DATE_COMPLETE',
    'PAYMENT DATE': 'DATE_PAID',
    'LICENSE APPROVED FOR ISSUANCE': 'DATE_APPROVED',
    'APPLICATION CREATED DATE': 'DATE_CREATED'
    }
licenses.rename(naming, axis=1, inplace=True)

# exclude record ID (redundant)
licenses.drop('RECORD_ID', axis=1, inplace=True)
licenses.drop('CONDITIONAL APPROVAL', axis=1, inplace=True)

In [5]:
# fix conflicting license number
index = licenses.index[licenses.LICENSE_ID==1237028][0]
licenses.loc[index, 'LICENSE_NUMBER'] = 2022

In [6]:
# split locations into separate database
location_vars = ['NAME', 'DBA', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 
                 'WARD', 'PRECINCT', 'WARD PRECINCT', 'POLICE DISTRICT', 
                 'LATITUDE', 'LONGITUDE', 'LOCATION', 'SSA']
locations = licenses.groupby(['ACCOUNT_ID', 'SITE_ID'])[location_vars].first()
locations.sort_index(level=0, inplace=True)
locations.drop(['WARD PRECINCT', 'LOCATION'], axis=1, inplace=True)
licenses.drop(labels=location_vars, axis=1, inplace=True)

In [7]:
# make license ID the new index
licenses.set_index('LICENSE_ID', inplace=True)

In [8]:
# reorder collumns
column_order = ['ACCOUNT_ID', 'SITE_ID', 'LICENSE_NUMBER', 'STATUS', 'APPLICATION_TYPE',
               'CODE', 'CODE_DESCRIPTION', 'ACTIVITY_ID', 'ACTIVITY','DATE_START', 'DATE_END',
                'DATE_CREATED', 'DATE_COMPLETE', 'DATE_PAID', 'DATE_APPROVED', 'DATE_ISSUED' , 'DATE_CHANGED']
licenses = licenses[column_order]

In [9]:
# assign start date to each license
licenses['START'] = licenses.DATE_START

# fill in start dates with issued date (where unavailable)
no_start = licenses[licenses.DATE_START.isnull()].index
licenses.loc[no_start, 'START'] = licenses.loc[no_start, 'DATE_ISSUED']

In [10]:
# store cancellations
cancellations = licenses[licenses.STATUS=='AAC']
revokations = licenses[licenses.STATUS=='REV']

In [11]:
# set license end date
licenses['END'] = licenses.DATE_END

# fix 90 day term length for shared kitchens
short_terms = licenses.ACTIVITY_ID=='805'
licenses.loc[short_terms, 'END'] = licenses.loc[short_terms, 'START'] + pd.Timedelta(value=90, unit='D')

# fix incorrect end date
licenses.loc[1300260, 'END'] = licenses.loc[1300260, 'START'] + pd.Timedelta(value=1, unit='Y')

# assign end date to terminated licenses
terminated_ind = licenses.STATUS.isin(['AAC', 'REV'])
licenses.loc[terminated_ind, 'END'] = licenses.loc[terminated_ind, 'DATE_CHANGED']
licenses.loc[terminated_ind, 'START'] = licenses.loc[terminated_ind, 'DATE_CHANGED']

In [12]:
# exclude name/location changes (temporary)
changes = licenses[~licenses.APPLICATION_TYPE.isin(['RENEW', 'ISSUE'])]
licenses = licenses[licenses.APPLICATION_TYPE.isin(['RENEW', 'ISSUE'])]

In [13]:
# exclude licenses whose end date falls before start date
incorrect_dates = (licenses.END - licenses.START) < pd.Timedelta(0)
licenses = licenses[~incorrect_dates]

# exclude licences whose terms ended before 2001
licenses = licenses[licenses.END.apply(lambda x: x.year >= 2001)]

# exclude reapplications/inquiries
licenses = licenses[licenses.STATUS.isin(['AAI', 'AAC', 'REV'])]

In [14]:
# sort by start date
licenses.sort_values(by='START', inplace=True)

In [ ]:
# save to HDF
licenses.to_hdf('../data/processed/licenses.hdf', 'licenses', mode='a')
changes.to_hdf('../data/processed/licenses.hdf', 'changes', mode='a')
cancellations.to_hdf('../data/processed/licenses.hdf', 'cancellations', mode='a')
revokations.to_hdf('../data/processed/licenses.hdf', 'revokations', mode='a')
locations.to_hdf('../data/processed/locations.hdf', 'locations', mode='a')

# ASSEMBLE INDIVIDUAL BUSINESS LICENSE HISTORIES

In [68]:
# compile license history dataframe
license_history, failed = [], []
for (indices, records) in licenses.groupby(['ACCOUNT_ID', 'LICENSE_NUMBER']):   
    try:
        license_history.append(License(records).aggregate())
    except:
        failed.append(indices)        
license_history = pd.DataFrame(license_history)

# reindex and set column order
license_history.set_index('LICENSE_NUMBER', inplace=True)
column_order = ['ACCOUNT_ID', 'SITE_ID', 'STATUS', 'START', 'END', 'ORIGINAL', 'CONTINUOUS', 'CODE', 'CODE_DESCRIPTION', 'ACTIVITY', 'ACTIVITY_ID']
license_history = license_history[column_order]

In [117]:
# set end date to today for active licenses
license_history.loc[license_history.STATUS == 'ACTIVE', 'END'] = pd.Timestamp.today()
license_history['LIFESPAN'] = license_history.END - license_history.START

In [118]:
# convert activity strings to tuples
license_history['ACTIVITY_ID'] = license_history.ACTIVITY_ID.apply(lambda x: tuple(x.split(' | ')) if type(x) == str else tuple())
license_history['ACTIVITY'] = license_history.ACTIVITY.apply(lambda x: tuple(x.split(' | ')) if type(x) == str else tuple())

In [119]:
# map license codes to descriptions
license_history.CODE = license_history.CODE.astype(str)
codebook = dict(zip(license_history['CODE'].unique(), license_history['CODE_DESCRIPTION'].unique()))

In [121]:
# map activity names to activities
activitybook = {}
for ID, NAME in zip(license_history.ACTIVITY_ID.unique(), license_history.ACTIVITY.unique()):    
    for i, n in zip(ID, NAME):
        activitybook[i] = n

In [124]:
# remove descriptors
license_history.drop(['CODE_DESCRIPTION', 'ACTIVITY'], axis=1, inplace=True)

In [ ]:
# reset index
location_history.set_index(['ACCOUNT_ID', 'SITE_ID'], inplace=True)
location_history.sort_values(by='START', inplace=True)

In [142]:
# save preprocessed license history data
license_history.to_hdf('../data/processed/licenses.hdf', 'history', mode='a')
write_json('../data/processed/license_codebook.json', codebook)
write_json('../data/processed/activity_codebook.json', activitybook)

# LOAD BUSINESS LICENSE HISTORIES

In [7]:
# read preprocessed data (license history)
license_history = pd.read_hdf('../data/processed/licenses.hdf', 'history')
codebook = read_json('../data/processed/license_codebook.json')
activitybook = read_json('../data/processed/activity_codebook.json')

# ASSEMBLE BUSINESS LOCATION HISTORIES

In [13]:
# identify unique locations
location_data = license_history.groupby(['ACCOUNT_ID', 'SITE_ID'])

location_history = []
for index, records in location_data:
    location_history.append(Location(records).aggregate())
location_history = pd.DataFrame(location_history)

# set column order
column_order = ['ACCOUNT_ID', 'SITE_ID', 'ACTIVE', 'START', 'END', 'LIFESPAN', 'ORIGINAL', 'CONTINUOUS', 'NUM_LICENSES', 'ACTIVITIES', 'CODES']
location_history = location_history[column_order]

# set lifespan to days
location_history.LIFESPAN = location_history.LIFESPAN.apply(lambda x: x.days)

# SAVE
location_history.to_hdf('../data/processed/locations.hdf', 'history', mode='a')

# LOAD BUSINESS LOCATION HISTORIES

In [28]:
# load aggregate location data
location_history = pd.read_hdf('../data/processed/locations.hdf', 'history')

# read locations
locations = pd.read_hdf('../data/processed/locations.hdf', 'locations')